In [ ]:
#ran on server to extract basic information about annotations:
import glob

gff3s=glob.glob("./gff3/*")
#gff3s=["1.161.O.cds.gff3"]

out=open("cds.annotation.info.txt","w")

for g in gff3s:
    phage=g.split("/")[-1].replace("cds.gff3","")
    cog_ct=0
    pfam_ct=0
    ann=open(g).readlines()
    total=len(ann)
    for line in ann:
        if "COG" in line:
            cog_ct+=1
        if "Pfam" in line:
            pfam_ct+=1
    out.write(phage+"\t"+str(total)+"\t"+str(cog_ct)+"\t"+str(pfam_ct))
out.close()

In [1]:
%load_ext rpy2.ipython
%R library(dplyr);library(magrittr);library(ggplot2); library(reshape)

/Users/jmb/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: 
Attaching package: ‘dplyr’


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/Users/jmb/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/Users/jmb/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  res = super(Function, self).__call__(*new_args, **new_kwargs)
/Users/jmb/anaconda/lib/python2.7/site-packages/rpy2/robjects/functions.py:106: UserWarning: Error in library(reshape) : there is no package called ‘reshape’

  res = super(Function, self).__call__(*new_args, **new_kwargs)



Error in library(reshape) : there is no package called ‘reshape’


In [2]:
%%R
ai<-read.table("cds.annotation.info1.txt", header=TRUE, sep="\t")
head(ai)

     phage total_orfs total_cogs total_pfam hypotheticals
1 1.245.O.         94          8         13            68
2 1.082.O.         57         13         15            45
3 1.040.O.         21          1          2            19
4 1.043.O.         21          0          1            20
5 1.074.O.         64          8         12            55
6 1.293.O.         76          4         15            68


In [3]:
%%R
ai<-mutate(ai, frac_cogs=total_cogs/total_orfs, frac_pfam=total_pfam/total_orfs, frac_hyp=hypotheticals/total_orfs)

In [4]:
%%R 
head(ai)

     phage total_orfs total_cogs total_pfam hypotheticals  frac_cogs  frac_pfam
1 1.245.O.         94          8         13            68 0.08510638 0.13829787
2 1.082.O.         57         13         15            45 0.22807018 0.26315789
3 1.040.O.         21          1          2            19 0.04761905 0.09523810
4 1.043.O.         21          0          1            20 0.00000000 0.04761905
5 1.074.O.         64          8         12            55 0.12500000 0.18750000
6 1.293.O.         76          4         15            68 0.05263158 0.19736842
   frac_hyp
1 0.7234043
2 0.7894737
3 0.9047619
4 0.9523810
5 0.8593750
6 0.8947368


In [29]:
%%R
with(ai, mean(frac_cogs))%>%print
with(ai, mean(frac_pfam))%>%print
with(ai, mean(frac_hyp))%>%print
sums<-colSums(ai[2:8],)
print(sums)

[1] 0.1462707
[1] 0.2173152
[1] 0.7625658
   total_orfs    total_cogs    total_pfam hypotheticals     frac_cogs 
  12055.00000    1695.00000    2565.00000    9055.00000      22.08687 
    frac_pfam      frac_hyp 
     32.81459     115.14744 


### Looking into kegg annotations via comparison to TARA versus comparison directly to kegg:

In [73]:
%%R
tara<-read.table("./1.161.O.tara.txt", sep="\t", header=FALSE, na.strings=c("","NA"))
head(tara)
rownames(tara)<-tara$V1
colnames(tara)[c(1,7,8,9)]<-c("locus_tag","tara_egg","tara_kegg","tara_kfunc")
head(tara)
tann<-tara[,c(1,2,7,8,9)]
head(tann)

                locus_tag                  V2 tara_egg tara_kegg tara_kfunc
NVP1161O_119 NVP1161O_119 OM-RGC.v1.031044528     <NA>      <NA>       <NA>
NVP1161O_118 NVP1161O_118 OM-RGC.v1.029216919     <NA>      <NA>       <NA>
NVP1161O_223 NVP1161O_223 OM-RGC.v1.023443165     <NA>      <NA>       <NA>
NVP1161O_110 NVP1161O_110 OM-RGC.v1.005269655  COG4294    K13281       <NA>
NVP1161O_117 NVP1161O_117 OM-RGC.v1.036345776     <NA>      <NA>       <NA>
NVP1161O_224 NVP1161O_224 OM-RGC.v1.018155996 NOG82947      <NA>       <NA>


In [109]:
%%R
others<-read.csv("1.161.O.annotations.txt", sep="\t", header=TRUE, na.strings=c(""," ","NA"))
ocn<-colnames(others)
colnames(others)<-ocn[2:length(ocn)]

others$locus_tag<-rownames(others)
others$"NA"<-NULL
nrow(others)%>%print
print(tail(others))
oann<-others[,c(5,6,7,8,9,10,11,12)]
head(oann)

[1] 228
             type  start   stop strand            best_hit_annotation KEGG PFam
NVP1161O_223  cds 136065 136541      +          hypothetical protein  <NA> <NA>
NVP1161O_224  cds 136547 137098      + putative phage-related protein <NA> <NA>
NVP1161O_225  cds 137723 137424      -           hypothetical protein <NA> <NA>
NVP1161O_226  cds 137968 138540      +           hypothetical protein <NA> <NA>
NVP1161O_227  cds 138960 139349      +             predicted protein  <NA> <NA>
NVP1161O_228  cds 139446 140666      +           hypothetical protein <NA> <NA>
              COG            ACLAME               CVP                TARA
NVP1161O_223 <NA>              <NA> CAMPEP_0000014776 OM-RGC.v1.023443165
NVP1161O_224 <NA> protein:vir:96721 CAMPEP_0000014874 OM-RGC.v1.019388685
NVP1161O_225 <NA>              <NA>              <NA>                <NA>
NVP1161O_226 <NA>              <NA>              <NA>                <NA>
NVP1161O_227 <NA>              <NA> CAMPEP_0000011814 OM-RGC.v

In [110]:
%%R
anns<-merge(tann, oann, by="locus_tag", all.x=TRUE, all.y=TRUE)
str(anns)

'data.frame':	228 obs. of  12 variables:
 $ locus_tag          : Factor w/ 228 levels "NVP1161O_003",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ V2                 : Factor w/ 88 levels "OM-RGC.v1.000339379",..: 16 6 40 15 61 18 39 22 52 35 ...
 $ tara_egg           : Factor w/ 22 levels "COG0207","COG0208",..: 4 14 NA NA NA 16 NA NA NA NA ...
 $ tara_kegg          : Factor w/ 8 levels "K00287","K00526",..: 3 NA NA NA NA NA NA NA NA NA ...
 $ tara_kfunc         : Factor w/ 4 levels "M00035","M00050,M00053,M00049",..: 1 NA NA NA NA NA NA NA NA NA ...
 $ best_hit_annotation: Factor w/ 28 levels "5' nucleotidase, deoxy (Pyrimidine), cytosolic type C protein (NT5C)",..: 2 17 17 17 17 15 12 17 17 12 ...
 $ KEGG               : Factor w/ 2 levels "K00287","K00560": NA NA NA NA NA NA NA NA NA NA ...
 $ PFam               : Factor w/ 14 levels "PF00145","PF00186",..: 1 NA NA NA NA 10 NA NA NA NA ...
 $ COG                : Factor w/ 5 levels "COG0207","COG0216",..: NA NA NA NA NA NA NA NA NA NA ...
 $ ACL

In [111]:
%%R
kegg_compare<-anns[,c(1,6,4,7)]
kegg_compare%>%.[which(.$tara_kegg!="<NA>"|.$KEGG!="<NA>"),]%>%write.table(.,file="1.161.O.kegg.comp.txt",sep="\t", quote=FALSE, row.names=FALSE)

In [112]:
%%R
cog_compare<-anns[,c(1,6,3,9)]
cog_compare%>%.[which(.[,3]!="<NA>"|.[,4]!="<NA>"),]%>%write.table(.,file="1.161.O.cog.comp.txt", sep="\t", quote=FALSE, row.names=FALSE)

In [119]:
%%R
anns%>%.[which(.[,c(3:11)]!="<NA>"),]%>%write.table(.,file="1.161.O.any.annote.txt", sep="\t",quote=FALSE, row.names=FALSE)

In [120]:
%%R
print(nrow(others))
anns%>%.[which(.[,c(3:11)]!="<NA>"),]%>%nrow%>%print

[1] 228
[1] 435
